In [8]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [9]:
code = """
#include<iostream>
#include<math.h>
#include<cuda_runtime.h>
#include<cstdlib>


#define n 8

using namespace std;

__global__ void minimum(int *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    //printf("No of threads = %d", number_of_threads);
    
    while(number_of_threads>0) {
        if(tid < number_of_threads) {
            int first = tid*step_size*2;
            int second = first + step_size;
            if(input[second] < input[first])
              input[first] = input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

__global__ void maximum(int *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads>0) {
        if(tid < number_of_threads) {
            int first = tid*step_size*2;
            int second = first + step_size;
            if(input[second] > input[first])
              input[first] = input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

__global__ void sum(int *input) {
    const int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads > 0) {
        if(tid < number_of_threads) {
            int first = tid * step_size * 2;
            int second = first + step_size;
            
            input[first] += input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
       
    }
}

__global__ void mean_diff_sq(float *input, float mean) {
    input[threadIdx.x] -= mean;
    input[threadIdx.x] *= input[threadIdx.x];
}

__global__ void sum_floats(float *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads > 0) {
        if(tid < number_of_threads) {
            int first = tid * step_size * 2;
            int second = first + step_size;
            
            input[first] += input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
       
    }
}

void copy_int_to_float(float *dest, int *src, int size){
    for(int i=0; i<size; i++)
        dest[i] = float(src[i]);
}

void random_ints(int *input, int size) {
    for(int i=0; i<size; i++)  {
        input[i] = rand()%100;
        cout<<input[i]<<"  ";   
    }
    cout<<endl;

}

int main() {
    //int n=8;
    int size = n*sizeof(int); //calculate no. of bytes for array
        
    int *arr;
    int *arr_d, result;
   
    arr = (int *)malloc(size);
    random_ints(arr, n);
    
    cudaMalloc((void **)&arr_d, size);
    
    //MIN
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    
    minimum<<<1,n/2>>>(arr_d);
    
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    
    cout<<"The minimum element is "<<result<<endl;
      
       
    //MAX
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    
    maximum<<<1,n/2>>>(arr_d);
    
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    
    cout<<"The maximum element is "<<result<<endl;
    
    //SUM
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    
    sum<<<1,n/2>>>(arr_d);
    
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    
    cout<<"The sum is "<<result<<endl;
    
    //AVERAGE
    
    float mean = float(result)/n;
    cout<<"The mean is "<<mean<<endl;
    
    //STANDARD DEVIATION
    float *arr_float;
    float *arr_std, stdValue;
    
    arr_float = (float *)malloc(n*sizeof(float));
    cudaMalloc((void **)&arr_std, n*sizeof(float));
    
    copy_int_to_float(arr_float, arr, n);
    
    cudaMemcpy(arr_std, arr_float, n*sizeof(float), cudaMemcpyHostToDevice);
    
    mean_diff_sq <<<1,n>>>(arr_std, mean);
    sum_floats<<<1,n/2>>>(arr_std);
    
    cudaMemcpy(&stdValue, arr_std, sizeof(float), cudaMemcpyDeviceToHost);
    
    
    stdValue = stdValue / n;
    cout<<"The variance is "<<stdValue<<endl;
    stdValue = sqrt(stdValue);
    
    cout<<"The standard deviation is "<<stdValue<<endl;
    
    cudaFree(arr_d);
           
    return 0;
}
"""

In [10]:
text_file = open("ass1.cu", "w")
text_file.write(code)
text_file.close()

In [11]:
!nvcc ass1.cu

In [12]:
!./a.out

83  86  77  15  93  35  86  92  
The minimum element is 15
The maximum element is 93
The sum is 567
The mean is 70.875
The variance is 748.359
The standard deviation is 27.3562


In [13]:
!nvprof ./a.out

83  86  77  15  93  35  86  92  
==190== NVPROF is profiling process 190, command: ./a.out
The minimum element is 15
The maximum element is 93
The sum is 567
The mean is 70.875
The variance is 748.359
The standard deviation is 27.3562
==190== Profiling application: ./a.out
==190== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   22.85%  6.6240us         4  1.6560us  1.6320us  1.6640us  [CUDA memcpy DtoH]
                   20.86%  6.0480us         4  1.5120us  1.3760us  1.8880us  [CUDA memcpy HtoD]
                   13.13%  3.8080us         1  3.8080us  3.8080us  3.8080us  minimum(int*)
                   12.36%  3.5840us         1  3.5840us  3.5840us  3.5840us  maximum(int*)
                   10.93%  3.1680us         1  3.1680us  3.1680us  3.1680us  sum_floats(float*)
                   10.93%  3.1680us         1  3.1680us  3.1680us  3.1680us  sum(int*)
                    8.94%  2.5920us         1  2.5920us  2.59